# E-commerce Data Analysis: Comprehensive Business Intelligence Report

**Author:** Data Analysis Team  
**Last Updated:** 2023  
**Version:** 2.0 (Refactored)

## Executive Summary

This notebook provides a comprehensive analysis of e-commerce business performance, focusing on revenue trends, product category performance, geographic distribution, and customer satisfaction metrics. The analysis is designed to be configurable and reusable for different time periods and business scenarios.

## Table of Contents

1. [Business Objectives](#business-objectives)
2. [Data Configuration](#data-configuration)
3. [Data Loading & Processing](#data-loading--processing)
4. [Data Dictionary](#data-dictionary)
5. [Revenue Performance Analysis](#revenue-performance-analysis)
6. [Product Category Analysis](#product-category-analysis)
7. [Geographic Distribution Analysis](#geographic-distribution-analysis)
8. [Customer Experience Analysis](#customer-experience-analysis)
9. [Key Findings & Recommendations](#key-findings--recommendations)

---

## 1. Business Objectives

This analysis aims to answer key business questions:

**Revenue Performance:**
- How has revenue performed year-over-year?
- What are the monthly growth trends?
- How has average order value changed?

**Product Performance:**
- Which product categories drive the most revenue?
- How concentrated is revenue across categories?

**Geographic Insights:**
- Which states generate the highest revenue?
- How is revenue distributed geographically?

**Customer Experience:**
- What is our customer satisfaction level?
- How does delivery speed impact customer ratings?
- What percentage of orders are delivered quickly?

## 2. Data Configuration

**Configure your analysis parameters below:**

In [ ]:
# Analysis Configuration
ANALYSIS_YEAR = 2023        # Primary year to analyze
COMPARISON_YEAR = 2022      # Comparison year for growth metrics (set to None to disable)
ANALYSIS_MONTH = None       # Specific month to analyze (1-12, or None for full year)
ORDER_STATUS = 'delivered'  # Order status to include ('delivered', 'shipped', etc.)
DATA_PATH = 'ecommerce_data/'  # Path to data files

# Visualization settings
COLOR_PALETTE = 'viridis'
FIGURE_SIZE = (12, 6)

print(f"Analysis configured for:")
print(f"  Primary Year: {ANALYSIS_YEAR}")
print(f"  Comparison Year: {COMPARISON_YEAR if COMPARISON_YEAR else 'None'}")
print(f"  Month Filter: {ANALYSIS_MONTH if ANALYSIS_MONTH else 'Full Year'}")
print(f"  Order Status: {ORDER_STATUS}")

## 3. Data Loading & Processing

Loading and processing e-commerce datasets using our custom data loading and metrics calculation modules.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import warnings

# Import custom modules
from data_loader import EcommerceDataLoader, load_and_process_data, get_data_summary
from business_metrics import BusinessMetricsCalculator, MetricsVisualizer

# Configure display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
warnings.filterwarnings('ignore')

print("Libraries and modules imported successfully!")

In [ ]:
# Load and process all datasets
loader, processed_data = load_and_process_data(DATA_PATH)

# Display data summary
get_data_summary(processed_data)

In [ ]:
# Create filtered sales dataset based on configuration
sales_data = loader.create_sales_dataset(
    year_filter=[ANALYSIS_YEAR, COMPARISON_YEAR] if COMPARISON_YEAR else [ANALYSIS_YEAR],
    month_filter=ANALYSIS_MONTH,
    status_filter=ORDER_STATUS
)

# Get supporting datasets
products_df = loader.get_product_categories()
customers_df = loader.get_customer_geography()
reviews_df = loader.get_review_scores()

print(f"\nAnalysis dataset created with {len(sales_data):,} records")
print(f"Date range: {sales_data['order_purchase_timestamp'].min().date()} to {sales_data['order_purchase_timestamp'].max().date()}")

## 4. Data Dictionary

**Key Business Terms and Metrics:**

| Term | Definition |
|------|------------|
| **Revenue** | Total sum of delivered order item prices (excludes canceled/returned orders) |
| **Average Order Value (AOV)** | Average total value per order (sum of all items in an order) |
| **Order** | A single purchase transaction that may contain multiple items |
| **Delivered Orders** | Orders with status 'delivered' (completed transactions) |
| **Delivery Speed** | Days between order purchase and customer delivery |
| **Review Score** | Customer satisfaction rating on 1-5 scale (5 = highest satisfaction) |
| **Product Category** | Business classification of products (13 categories total) |
| **Revenue Growth** | Year-over-year percentage change in total revenue |
| **Monthly Growth Trend** | Average month-over-month revenue change |

**Key Dataset Relationships:**
- Orders ↔ Order Items (one-to-many: one order can have multiple items)
- Orders ↔ Reviews (one-to-one: each order can have one review)
- Orders ↔ Customers (many-to-one: customers can have multiple orders)
- Order Items ↔ Products (many-to-one: same product can be in multiple orders)

## 5. Revenue Performance Analysis

Comprehensive analysis of revenue trends, growth rates, and order patterns.

In [ ]:
# Initialize business metrics calculator
metrics_calculator = BusinessMetricsCalculator(sales_data)

# Calculate revenue metrics
revenue_metrics = metrics_calculator.calculate_revenue_metrics(ANALYSIS_YEAR, COMPARISON_YEAR)

# Display key revenue metrics
print(f"REVENUE PERFORMANCE SUMMARY - {ANALYSIS_YEAR}")
print("=" * 50)
print(f"Total Revenue: ${revenue_metrics['total_revenue']:,.2f}")
print(f"Total Orders: {revenue_metrics['total_orders']:,}")
print(f"Total Items Sold: {revenue_metrics['total_items']:,}")
print(f"Average Order Value: ${revenue_metrics['average_order_value']:.2f}")
print(f"Average Item Price: ${revenue_metrics['average_item_price']:.2f}")

if COMPARISON_YEAR and 'revenue_growth_rate' in revenue_metrics:
    print(f"\nYEAR-OVER-YEAR COMPARISON ({COMPARISON_YEAR} vs {ANALYSIS_YEAR}):")
    print(f"Revenue Growth: {revenue_metrics['revenue_growth_rate']:+.1f}%")
    print(f"Order Growth: {revenue_metrics['order_growth_rate']:+.1f}%")
    print(f"AOV Growth: {revenue_metrics['aov_growth_rate']:+.1f}%")
    
print(f"\nMONTHLY TRENDS:")
print(f"Average Monthly Growth: {revenue_metrics['monthly_growth_trend']:+.2f}%")

In [ ]:
# Create comprehensive revenue analysis report
comprehensive_report = metrics_calculator.generate_comprehensive_report(
    current_year=ANALYSIS_YEAR,
    previous_year=COMPARISON_YEAR,
    products_df=products_df,
    customers_df=customers_df,
    reviews_df=reviews_df
)

# Initialize visualizer
visualizer = MetricsVisualizer(comprehensive_report, COLOR_PALETTE)

# Create revenue trend visualization
revenue_fig = visualizer.plot_revenue_trend(FIGURE_SIZE)
plt.show()

# Monthly revenue data table
monthly_revenue_df = pd.DataFrame(list(revenue_metrics['monthly_revenue'].items()), 
                                 columns=['Month', 'Revenue'])
monthly_revenue_df['Revenue_Formatted'] = monthly_revenue_df['Revenue'].apply(lambda x: f'${x:,.0f}')
print("\nMonthly Revenue Breakdown:")
print(monthly_revenue_df[['Month', 'Revenue_Formatted']].to_string(index=False))

## 6. Product Category Analysis

Analysis of product category performance, revenue concentration, and market share distribution.

In [ ]:
# Calculate product performance metrics
product_metrics = metrics_calculator.calculate_product_metrics(products_df)

print(f"PRODUCT CATEGORY PERFORMANCE - {ANALYSIS_YEAR}")
print("=" * 50)
print(f"Total Product Categories: {product_metrics['total_categories']}")
print(f"Top 5 Categories Revenue Concentration: {product_metrics['revenue_concentration']:.1f}%")

print("\nTop 10 Categories by Revenue:")
top_categories = product_metrics['top_categories'][['total_revenue', 'revenue_share_pct', 'total_orders']]
top_categories['total_revenue'] = top_categories['total_revenue'].apply(lambda x: f'${x:,.0f}')
top_categories['revenue_share_pct'] = top_categories['revenue_share_pct'].apply(lambda x: f'{x:.1f}%')
print(top_categories.to_string())

In [ ]:
# Create product category performance visualization
category_fig = visualizer.plot_category_performance(top_n=10, figsize=(12, 8))
plt.show()

# Additional category insights
all_categories = product_metrics['category_performance']
print(f"\nCategory Performance Insights:")
print(f"Highest AOV Category: {all_categories.loc[all_categories['total_revenue'].div(all_categories['total_orders']).idxmax()].name}")
print(f"Most Orders Category: {all_categories['total_orders'].idxmax()}")
print(f"Most Diverse Category (most unique products): {all_categories['unique_products'].idxmax()}")

## 7. Geographic Distribution Analysis

State-level revenue analysis and geographic market penetration insights.

In [ ]:
# Calculate geographic performance metrics
geographic_metrics = metrics_calculator.calculate_geographic_metrics(customers_df)

print(f"GEOGRAPHIC PERFORMANCE - {ANALYSIS_YEAR}")
print("=" * 50)
print(f"Total States with Sales: {geographic_metrics['total_states']}")
print(f"Top 10 States Revenue Concentration: {geographic_metrics['geographic_concentration']:.1f}%")

print("\nTop 10 States by Revenue:")
top_states = geographic_metrics['top_states'][['total_revenue', 'total_orders', 'avg_order_value', 'revenue_share_pct']]
top_states['total_revenue'] = top_states['total_revenue'].apply(lambda x: f'${x:,.0f}')
top_states['avg_order_value'] = top_states['avg_order_value'].apply(lambda x: f'${x:.0f}')
top_states['revenue_share_pct'] = top_states['revenue_share_pct'].apply(lambda x: f'{x:.1f}%')
print(top_states.to_string())

In [ ]:
# Create interactive geographic distribution map
geographic_fig = visualizer.plot_geographic_distribution()
geographic_fig.show()

# Geographic insights
all_states = geographic_metrics['state_performance']
print(f"\nGeographic Performance Insights:")
print(f"Highest Revenue State: {all_states['total_revenue'].idxmax()}")
print(f"Highest AOV State: {all_states['avg_order_value'].idxmax()}")
print(f"Most Active State (most orders): {all_states['total_orders'].idxmax()}")
print(f"Revenue per Customer Leader: {all_states['revenue_per_customer'].idxmax()}")

## 8. Customer Experience Analysis

Customer satisfaction, delivery performance, and correlation between service quality and ratings.

In [ ]:
# Calculate customer satisfaction and delivery metrics
experience_metrics = metrics_calculator.calculate_customer_satisfaction_metrics(reviews_df)

print(f"CUSTOMER EXPERIENCE PERFORMANCE - {ANALYSIS_YEAR}")
print("=" * 50)
print(f"Total Reviews Analyzed: {experience_metrics['total_reviews']:,}")

print(f"\nCUSTOMER SATISFACTION:")
satisfaction = experience_metrics['satisfaction']
print(f"Average Review Score: {satisfaction['average_review_score']:.2f}/5.0")
print(f"High Satisfaction Rate (4-5 stars): {satisfaction['high_satisfaction_rate']:.1f}%")
print(f"Low Satisfaction Rate (1-2 stars): {satisfaction['low_satisfaction_rate']:.1f}%")

print(f"\nDELIVERY PERFORMANCE:")
delivery = experience_metrics['delivery']
print(f"Average Delivery Time: {delivery['average_delivery_days']:.1f} days")
print(f"Median Delivery Time: {delivery['median_delivery_days']:.0f} days")
print(f"Fast Delivery Rate (≤3 days): {delivery['fast_delivery_rate']:.1f}%")
print(f"Slow Delivery Rate (>10 days): {delivery['slow_delivery_rate']:.1f}%")

print(f"\nDELIVERY SPEED vs SATISFACTION CORRELATION:")
for speed_category, avg_score in experience_metrics['delivery_satisfaction_correlation'].items():
    print(f"{speed_category}: {avg_score:.2f}/5.0 average rating")

In [ ]:
# Create customer satisfaction distribution visualization
satisfaction_fig = visualizer.plot_satisfaction_distribution(figsize=(10, 6))
plt.show()

# Detailed satisfaction breakdown
satisfaction_dist = experience_metrics['satisfaction']['review_score_distribution'] * 100
print("\nDetailed Review Score Distribution:")
for score, percentage in satisfaction_dist.items():
    stars = '★' * int(score) + '☆' * (5 - int(score))
    print(f"{score} stars {stars}: {percentage:.1f}%")

## 9. Key Findings & Recommendations

Executive summary of insights and strategic recommendations based on the analysis.

In [ ]:
# Print comprehensive business summary
metrics_calculator.print_summary_report(comprehensive_report)

# Generate strategic insights
print("\n" + "="*60)
print("STRATEGIC INSIGHTS & RECOMMENDATIONS")
print("="*60)

# Revenue insights
revenue = comprehensive_report['revenue_metrics']
if 'revenue_growth_rate' in revenue:
    if revenue['revenue_growth_rate'] < 0:
        print(f"\n📉 REVENUE CONCERN:")
        print(f"   Revenue declined {abs(revenue['revenue_growth_rate']):.1f}% year-over-year")
        print(f"   Recommend investigating market conditions and competitive landscape")
    else:
        print(f"\n📈 REVENUE GROWTH:")
        print(f"   Revenue increased {revenue['revenue_growth_rate']:.1f}% year-over-year")
        print(f"   Continue current growth strategies and identify scaling opportunities")

# Product insights
if 'product_metrics' in comprehensive_report:
    products = comprehensive_report['product_metrics']
    top_category = products['category_performance'].index[0]
    print(f"\n🏆 PRODUCT FOCUS:")
    print(f"   '{top_category}' is the top revenue category")
    print(f"   Top 5 categories drive {products['revenue_concentration']:.1f}% of revenue")
    if products['revenue_concentration'] > 70:
        print(f"   Consider diversifying product portfolio to reduce concentration risk")

# Customer experience insights
if 'customer_experience_metrics' in comprehensive_report:
    experience = comprehensive_report['customer_experience_metrics']
    avg_rating = experience['satisfaction']['average_review_score']
    avg_delivery = experience['delivery']['average_delivery_days']
    
    print(f"\n🎯 CUSTOMER EXPERIENCE:")
    print(f"   Average satisfaction: {avg_rating:.1f}/5.0")
    if avg_rating >= 4.0:
        print(f"   Strong customer satisfaction maintained")
    else:
        print(f"   Customer satisfaction below 4.0 - investigate quality issues")
        
    print(f"   Average delivery time: {avg_delivery:.1f} days")
    if avg_delivery > 7:
        print(f"   Delivery speed optimization could improve satisfaction scores")
        print(f"   Fast delivery (1-3 days) correlates with higher ratings")

print(f"\n" + "="*60)
print(f"Analysis completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Configuration: {ANALYSIS_YEAR} analysis with {ORDER_STATUS} orders only")
print("="*60)

---

## Conclusion

This refactored analysis provides a comprehensive, configurable framework for e-commerce business intelligence. The modular structure allows for:

- **Easy reconfiguration** for different time periods
- **Reusable code modules** for consistent analysis
- **Professional visualizations** for business presentation
- **Automated insights generation** for strategic decision-making

### Next Steps

1. **Regular Monitoring**: Run this analysis monthly/quarterly for trend tracking
2. **Segmentation**: Extend analysis to include customer segmentation
3. **Forecasting**: Add predictive analytics for revenue forecasting
4. **Automation**: Consider automated report generation and distribution

### Technical Notes

- All pandas warnings have been addressed using proper DataFrame operations
- Code is modularized in `data_loader.py` and `business_metrics.py`
- Analysis is fully configurable via parameters at the top of the notebook
- Visualizations use consistent business-oriented styling

---

**Report Generated:** {datetime.now().strftime('%B %d, %Y at %I:%M %p')}  
**Framework Version:** 2.0 (Refactored)  
**Contact:** Data Analysis Team